In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [29]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-23 20:00:00+00:00


In [30]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [31]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-23 15:03:44,078 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-23 15:03:44,081 INFO: Initializing external client
2025-03-23 15:03:44,081 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-23 15:03:45,127 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214685


In [32]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.92s) 


In [33]:
df["pickup_hour"].max()

Timestamp('2025-03-23 19:00:00+0000', tz='Etc/UTC')

In [34]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,176,0.0,2025-03-06 18:00:00+00:00
1,234,131.0,2025-03-06 18:00:00+00:00
2,31,0.0,2025-03-06 18:00:00+00:00
3,255,1.0,2025-03-06 18:00:00+00:00
4,43,71.0,2025-03-06 18:00:00+00:00
...,...,...,...
88387,233,87.0,2025-03-23 19:00:00+00:00
88388,183,0.0,2025-03-23 19:00:00+00:00
88389,82,1.0,2025-03-23 19:00:00+00:00
88390,68,162.0,2025-03-23 19:00:00+00:00


In [35]:
#a.info()

In [36]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-23 15:03:48,147 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-23 15:03:48,149 INFO: Initializing external client
2025-03-23 15:03:48,149 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-23 15:03:48,870 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214685
Current UTC time: 2025-03-23 19:03:48.147019+00:00
Next hour: 2025-03-23 20:00:00+00:00
Querying for date range: 2025-03-23 to 2025-03-24
Filtering for hour: 2025-03-23 20:00
Found 0 records


In [37]:
now = datetime.now(timezone.utc)

In [38]:
predictions

,pickup_location_id,predicted_demand,pickup_hour


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88392 entries, 0 to 88391
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  88392 non-null  int32                  
 1   predicted_demand    88392 non-null  float64                
 2   pickup_hour         88392 non-null  datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 1.7 MB


In [40]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [41]:
dt = current_date.ceil('h')


In [42]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-23 15:03:50,008 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-23 15:03:50,009 INFO: Initializing external client
2025-03-23 15:03:50,009 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-23 15:03:50,670 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214685
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.91s) 


In [43]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.78s) 


,pickup_location_id,predicted_demand,pickup_hour


In [44]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,176,0.0,2025-03-06 18:00:00+00:00
1,234,131.0,2025-03-06 18:00:00+00:00
2,31,0.0,2025-03-06 18:00:00+00:00
3,255,1.0,2025-03-06 18:00:00+00:00
4,43,71.0,2025-03-06 18:00:00+00:00
...,...,...,...
88387,233,87.0,2025-03-23 19:00:00+00:00
88388,183,0.0,2025-03-23 19:00:00+00:00
88389,82,1.0,2025-03-23 19:00:00+00:00
88390,68,162.0,2025-03-23 19:00:00+00:00


In [45]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=2)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [46]:
predictions = fetch_next_hour_predictions()

2025-03-23 15:03:54,918 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-23 15:03:54,920 INFO: Initializing external client
2025-03-23 15:03:54,920 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-23 15:03:55,670 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214685
Current UTC time: 2025-03-23 19:03:54.918156+00:00
Next hour: 2025-03-23 21:00:00+00:00
Found 0 records


In [47]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-03-23 15:03:56,711 INFO: Closing external client and cleaning up certificates.


Connection closed.
2025-03-23 15:03:56,713 INFO: Initializing external client
2025-03-23 15:03:56,713 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-23 15:03:57,404 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214685


In [48]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.82s) 


In [49]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,176,0.0,2025-03-06 18:00:00+00:00
1,234,131.0,2025-03-06 18:00:00+00:00
2,31,0.0,2025-03-06 18:00:00+00:00
3,255,1.0,2025-03-06 18:00:00+00:00
4,43,71.0,2025-03-06 18:00:00+00:00
...,...,...,...
88387,233,87.0,2025-03-23 19:00:00+00:00
88388,183,0.0,2025-03-23 19:00:00+00:00
88389,82,1.0,2025-03-23 19:00:00+00:00
88390,68,162.0,2025-03-23 19:00:00+00:00


In [50]:
df["pickup_hour"].max()

Timestamp('2025-03-23 19:00:00+0000', tz='Etc/UTC')

In [51]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-23 20:00:00+00:00


In [52]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
